In [8]:

using CSV, DataFrames, Plots, Random, Statistics, LinearAlgebra, JuMP, Gurobi, StatsBase, Dates, GLM, CategoricalArrays, MathOptInterface

In [9]:
df_train=DataFrame(CSV.File("train_final.csv"));
df_test=DataFrame(CSV.File("test_final.csv"));
capacities=DataFrame(CSV.File("capacities.csv"));
initial_stock=DataFrame(CSV.File("stock.csv"));

In [43]:
## Search for hyperparameters for a given station and hour

seed = 1504
Random.seed!(seed)

station_focus = df_train[1, :loc_id]  # adjust according to desired station
hour_focus = df_train[1, :hour]       # adjust according to desired hour

train_focus = filter(row -> row.loc_id == station_focus && row.hour == hour_focus, df_train)

feature_cols = [:day_of_week, :temp, :humidity, :pressure, :windspeed, :precipitation, :holiday, :exam_period, :semester]
categorical_cols = [:day_of_week, :semester]

X_train_focus = select(train_focus, feature_cols)

for col in categorical_cols
    X_train_focus[!, col] = categorical(X_train_focus[:, col])
end

y_train_focus = convert(Vector{Float64}, train_focus[:, :net_demand])

grid = IAI.GridSearch(
    IAI.OptimalTreeRegressor(random_seed = seed),
    max_depth = [6, 8, 10],
    minbucket = [2, 5, 10],
    cp = [0.0, 1e-2, 0.1]
)
IAI.fit_cv!(grid, X_train_focus, y_train_focus, n_folds = 5)

best_model_focus = IAI.get_learner(grid)
best_params_focus = IAI.get_params(best_model_focus)
println("Best hyperparameters (loc_id=$(station_focus), hour=$(hour_focus)):")
for (k, v) in sort(collect(best_params_focus), by = first)
    println("  ", k, " = ", v)
end


Best hyperparameters (loc_id=4235775371103934, hour=0):
  checkpoint_dir = nothing
  checkpoint_file = nothing
  checkpoint_interval = 10
  checkpoint_max_files = nothing
  cp = 0.01
  cp_tuning_se_tolerance = 0.0
  criterion = mse
  fast_cumulative_support = true
  fast_num_support_iterations = 1
  fast_num_support_restarts = 0
  fast_test_intermediate_support = true
  hinge_epsilon = 0.001
  hyperplane_config = NamedTuple[]
  localsearch = true
  ls_bootstrap_samples = false
  ls_ignore_errors = false
  ls_max_hyper_iterations = 9223372036854775807
  ls_max_search_iterations = 9223372036854775807
  ls_num_categoric_restarts = 10
  ls_num_greedy_features = auto
  ls_num_hyper_restarts = 5
  ls_num_tree_restarts = 100
  ls_scan_reverse_split = false
  ls_warmstart_criterion = mse
  max_categoric_levels_before_warning = 10
  max_depth = 8
  minbucket = 5
  missingdatamode = none
  normalize_X = true
  normalize_y = true
  num_threads = nothing
  parallel_processes = nothing
  random_see

In [44]:
# Train an optimal tree for each hour and station
# Use hyperparameters max_depth = 8, minbucket = 5, cp = 1e-2 (found previously)

seed = 1504
Random.seed!(seed)

# Get unique stations and hours
unique_stations = unique(df_train[:, :loc_id])
unique_hours = unique(df_train[:, :hour])
sort!(unique_stations)
sort!(unique_hours)

println("Training trees for $(length(unique_stations)) stations and $(length(unique_hours)) hours...")
println("Total models to train: $(length(unique_stations) * length(unique_hours))")

# Dictionary to store trained models: models[(loc_id, hour)] = model
models = Dict{Tuple{Int64, Int64}, Any}()

# Feature columns
feature_cols = [:day_of_week, :temp, :humidity, :pressure, :windspeed, :precipitation, :holiday, :exam_period, :semester]
categorical_cols = [:day_of_week, :semester]

# Counter to show progress
model_count = 0
total_models = length(unique_stations) * length(unique_hours)

for station in unique_stations
    for hour in unique_hours
        model_count += 1
        
        # Filter training data for this station and hour
        train_subset = filter(row -> row.loc_id == station && row.hour == hour, df_train)
        
        # Skip if not enough data
        if nrow(train_subset) < 10
            continue
        end
        
        # Prepare features
        X_train_subset = select(train_subset, feature_cols)
        
        # Convert categorical columns
        for col in categorical_cols
            X_train_subset[!, col] = categorical(X_train_subset[:, col])
        end
        
        # Prepare target
        y_train_subset = convert(Vector{Float64}, train_subset[:, :net_demand])
        
        # Create and train model with optimal hyperparameters
        model = IAI.OptimalTreeRegressor(
            random_seed = seed,
            max_depth = 8,
            minbucket = 5,
            cp = 1e-2
        )
        
        IAI.fit!(model, X_train_subset, y_train_subset)
        
        # Save the model
        models[(station, hour)] = model
        
        # Show progress every 50 models
        if model_count % 50 == 0 || model_count == total_models
            println("Progress: $(model_count)/$(total_models) models trained ($(round(100 * model_count / total_models, digits=1))%)")
        end
    end
end

println("\nTraining completed. Total models trained: $(length(models))")
println("Models per station: approximately $(round(length(models) / length(unique_stations), digits=1))")

Training trees for 10 stations and 24 hours...


Total models to train: 240
Progress: 50/240 models trained (20.8%)
Progress: 100/240 models trained (41.7%)
Progress: 150/240 models trained (62.5%)
Progress: 200/240 models trained (83.3%)
Progress: 240/240 models trained (100.0%)

Training completed. Total models trained: 240
Models per station: approximately 24.0


In [ ]:

function solve_prescriptive_optimization(S, station_distributions, C, q, p_empty, p_full, N)
    """
    Solves the prescriptive optimization problem by minimizing:
    - Redistribution cost (bicycle movement)
    - Expected empty/full cost weighted by the provided distributions

    `station_distributions` must be a vector of length N where each element
    exposes `net demands` and `probabilities`. Each station can have a different
    number of demands and their probabilities are already aggregated.
    """

    if length(S) != N || length(C) != N || length(station_distributions) != N
        error("Vectors S and C must have length N")
    end

    demands_by_station = Vector{Vector{Float64}}(undef, N)
    probs_by_station = Vector{Vector{Float64}}(undef, N)
    for i in 1:N
        entry = station_distributions[i]
        if hasproperty(entry, :demands) && hasproperty(entry, :probabilities)
            demands = collect(Float64.(entry.demands))
            probs = collect(Float64.(entry.probabilities))
        elseif entry isa Dict
            demands = collect(Float64.(entry[:demands]))
            probs = collect(Float64.(entry[:probabilities]))
        else
            error("Each element must expose `demands` and `probabilities`.")
        end

        if length(demands) != length(probs)
            error("Demands and probabilities must have equal length (station $(i)).")
        elseif isempty(demands)
            error("Station $(i) does not have demand scenarios defined.")
        end
        probs_sum = sum(probs)
        if probs_sum <= 0
            error("The sum of probabilities must be positive (station $(i)).")
        end
        probs ./= probs_sum

        demands_by_station[i] = demands
        probs_by_station[i] = probs
    end

    model = Model(Gurobi.Optimizer)
    set_silent(model)

    # Define redistribution variables (x[i,j] = bikes from station i to station j)
    @variable(model, x[i=1:N, j=1:N] >= 0, Int)
    
    # Add constraint that cannot redistribute to itself
    for i in 1:N
        @constraint(model, x[i, i] == 0)
    end

    z_empty = Dict{Tuple{Int,Int}, VariableRef}()
    z_full = Dict{Tuple{Int,Int}, VariableRef}()
    for i in 1:N
        num_scenarios = length(demands_by_station[i])
        for k in 1:num_scenarios
            z_empty[(i,k)] = @variable(model, binary = true, base_name="z_empty_$(i)_$(k)")
            z_full[(i,k)] = @variable(model, binary = true, base_name="z_full_$(i)_$(k)")
        end
    end

    for i in 1:N
        # Calculate net redistribution: sum(x[j,i]) - sum(x[i,j]) for j != i
        net_redistribution = sum(x[j,i] for j in 1:N) - sum(x[i,j] for j in 1:N)
        stock_after_redist = S[i] + net_redistribution
        @constraint(model, stock_after_redist >= 0)
        @constraint(model, stock_after_redist <= C[i])
    end

    M = maximum(C) + 1
    for i in 1:N
        # Calculate net redistribution: sum(x[j,i]) - sum(x[i,j]) for j != i
        net_redistribution = sum(x[j,i] for j in 1:N) - sum(x[i,j] for j in 1:N)
        for k in 1:length(demands_by_station[i])
            demand_k = clamp(demands_by_station[i][k], -C[i], C[i])
            final_stock = S[i] + net_redistribution + demand_k
            @constraint(model, final_stock >= 1 - M * z_empty[(i,k)])
            @constraint(model, final_stock <= C[i] - 1 + M * z_full[(i,k)])
        end
    end

    # Calculate movement cost - build expression explicitly
    movement_cost = @expression(model, 0.0)
    for i in 1:N
        for j in 1:N
            movement_cost += q * x[i,j]
        end
    end
    
    # Calculate penalty cost - build expression explicitly
    penalty_cost = @expression(model, 0.0)
    for i in 1:N
        for k in 1:length(demands_by_station[i])
            penalty_cost += probs_by_station[i][k] * p_empty * z_empty[(i,k)]
            penalty_cost += probs_by_station[i][k] * p_full * z_full[(i,k)]
        end
    end

    @objective(model, Min, movement_cost + penalty_cost)

    optimize!(model)

    if termination_status(model) == MOI.OPTIMAL
        x_sol = zeros(Int, N, N)
        for i in 1:N
            for j in 1:N
                if i != j
                    x_sol[i,j] = round(Int, value(x[i,j]))
                end
            end
        end
        obj_val = objective_value(model)
        return x_sol, obj_val
    else
        return zeros(Int, N, N), Inf
    end
end

solve_prescriptive_optimization (generic function with 1 method)

In [41]:
# Function to calculate cost without redistribution
function calculate_cost_no_redistribution(S, d_real, C, p_empty, p_full, N)
    """
    Calculates the cost without redistribution using real demand
    """
    total_cost = 0.0
    
    for i in 1:N
        final_stock = S[i] + d_real[i]
        
        if final_stock <= 0
            total_cost += p_empty
        elseif final_stock >= C[i]
            total_cost += p_full
        end
    end
    
    return total_cost
end

# Function to calculate cost with redistribution using real demand
function calculate_cost_with_redistribution(S, x, d_real, C, q, p_empty, p_full, N)
    """
    Calculates the cost with redistribution using real demand
    """
    # Movement cost
    movement_cost = q * sum(x[i,j] for i in 1:N for j in 1:N if i != j)
    
    # Calculate final stock after redistribution and real demand
    penalty_cost = 0.0
    for i in 1:N
        net_redistribution = sum(x[j,i] - x[i,j] for j in 1:N if j != i)
        final_stock = S[i] + net_redistribution + d_real[i]
        
        if final_stock <= 0
            penalty_cost += p_empty
        elseif final_stock >= C[i]
            penalty_cost += p_full
        end
    end
    
    return movement_cost + penalty_cost
end

println("Evaluation functions defined")


Evaluation functions defined


In [42]:
# Function that given a test day and hour, for each station (of that day and time)
# determines which leaf of the corresponding tree it falls into. Then collects all 
# net_demands from train that fall in that leaf. Looks at repeated values and the 
# frequency of each one.
# Returns a vector of N elements (for each station), where each element is a vector 
# of tuples (net_demand, probability)

function get_station_distributions(day, hour, models, df_train, df_test, unique_stations, feature_cols, categorical_cols, C)
    """
    Gets the demand distributions for each station based on tree leaves.
    
    Parameters:
    - day: Test day
    - hour: Test hour
    - models: Dictionary of trained models models[(loc_id, hour)]
    - df_train: Training DataFrame
    - df_test: Test DataFrame
    - unique_stations: Vector of unique stations (sorted)
    - feature_cols: Feature columns
    - categorical_cols: Categorical columns
    - C: Vector of capacities per station (must have same length as unique_stations)
    
    Returns:
    - Vector of N elements, where each element is a vector of tuples (net_demand, probability)
    """
    N = length(unique_stations)
    station_distributions = Vector{Vector{Tuple{Float64, Float64}}}(undef, N)
    
    # Filter test data for this day and hour
    test_subset = filter(row -> row.day == day && row.hour == hour, df_test)
    
    # Process each station
    for (idx, station) in enumerate(unique_stations)
        # Get test row for this station
        test_row = filter(row -> row.loc_id == station, test_subset)
        
        if nrow(test_row) == 0
            # If no test data, use empty or uniform distribution
            station_distributions[idx] = [(0.0, 1.0)]
            continue
        end
        
        test_row = test_row[1, :]
        
        # Get model for this station and hour
        if !haskey(models, (station, hour))
            # If no model, use empty distribution
            station_distributions[idx] = [(0.0, 1.0)]
            continue
        end
        
        model = models[(station, hour)]
        
        # Prepare test features for this station
        X_test_station = DataFrame()
        for col in feature_cols
            X_test_station[!, col] = [test_row[col]]
        end
        
        # Convert categorical columns
        for col in categorical_cols
            X_test_station[!, col] = categorical(X_test_station[:, col])
        end
        
        # Filter training data for this station and hour
        train_subset = filter(row -> row.loc_id == station && row.hour == hour, df_train)
        
        if nrow(train_subset) == 0
            station_distributions[idx] = [(0.0, 1.0)]
            continue
        end
        
        # Prepare train features
        X_train_subset = select(train_subset, feature_cols)
        
        # Convert categorical columns
        for col in categorical_cols
            X_train_subset[!, col] = categorical(X_train_subset[:, col])
        end
        
        # Get the leaf that this test observation belongs to
        # IAI.apply returns a vector with a leaf id per row
        leaf_test = IAI.apply(model, X_test_station)
        leaf_idx = leaf_test[1]  # First (and only) test observation
        
        # Get leaf assignments for all train observations
        leaf_train = IAI.apply(model, X_train_subset)  # One leaf id per train row
        
        # Filter observations that fall in the same leaf
        same_leaf_mask = leaf_train .== leaf_idx
        train_same_leaf = train_subset[same_leaf_mask, :]
        
        if nrow(train_same_leaf) == 0
            # If no observations in the same leaf, use tree prediction
            capacity_i = C[idx]
            prediction = IAI.predict(model, X_test_station)[1]
            # Round and truncate the prediction
            prediction = round(Int, clamp(prediction, -capacity_i, capacity_i))
            station_distributions[idx] = [(Float64(prediction), 1.0)]
            continue
        end
        
        # Get net_demands from observations that fall in the same leaf
        net_demands_raw = convert(Vector{Float64}, train_same_leaf[:, :net_demand])
        
        # Round to integers and truncate between -C[idx] and C[idx]
        capacity_i = C[idx]
        net_demands = [Float64(round(Int, clamp(demand, -capacity_i, capacity_i))) for demand in net_demands_raw]
        
        # Count frequencies of each net_demand
        demand_counts = Dict{Float64, Int}()
        for demand in net_demands
            demand_counts[demand] = get(demand_counts, demand, 0) + 1
        end
        
        # Calculate probabilities
        total_count = length(net_demands)
        distribution = Vector{Tuple{Float64, Float64}}()
        
        for (demand, count) in demand_counts
            probability = count / total_count
            push!(distribution, (demand, probability))
        end
        
        # Sort by demand (optional, for consistency)
        sort!(distribution, by = x -> x[1])
        
        station_distributions[idx] = distribution
    end
    
    return station_distributions
end

# Function to get point predictions (simple predictions)
function get_point_predictions(day, hour, models, df_test, unique_stations, feature_cols, categorical_cols, C)
    """
    Gets point predictions for each station, rounded and truncated.
    Returns format for solve_prescriptive_optimization (single demand with probability 1).
    """
    N = length(unique_stations)
    point_predictions = Vector{Any}(undef, N)
    
    # Filter test data for this day and hour
    test_subset = filter(row -> row.day == day && row.hour == hour, df_test)
    
    for (idx, station) in enumerate(unique_stations)
        # Get test row for this station
        test_row = filter(row -> row.loc_id == station, test_subset)
        
        if nrow(test_row) == 0 || !haskey(models, (station, hour))
            point_predictions[idx] = (demands = [0.0], probabilities = [1.0])
            continue
        end
        
        test_row = test_row[1, :]
        model = models[(station, hour)]
        
        # Prepare test features
        X_test_station = DataFrame()
        for col in feature_cols
            X_test_station[!, col] = [test_row[col]]
        end
        
        # Convert categorical columns
        for col in categorical_cols
            X_test_station[!, col] = categorical(X_test_station[:, col])
        end
        
        # Get prediction
        prediction = IAI.predict(model, X_test_station)[1]
        
        # Round and truncate
        capacity_i = C[idx]
        prediction = Float64(round(Int, clamp(prediction, -capacity_i, capacity_i)))
        
        point_predictions[idx] = (demands = [prediction], probabilities = [1.0])
    end
    
    return point_predictions
end

# Auxiliary function to convert tuple format to format expected by solve_prescriptive_optimization
function convert_to_optimization_format(station_distributions)
    """
    Converts tuple format (net_demand, probability) to format expected by 
    solve_prescriptive_optimization (with demands and probabilities).
    """
    N = length(station_distributions)
    formatted_distributions = Vector{Any}(undef, N)
    
    for i in 1:N
        dist = station_distributions[i]
        demands = [d[1] for d in dist]  # First element of each tuple
        probabilities = [d[2] for d in dist]  # Second element of each tuple
        
        # Create a NamedTuple with demands and probabilities
        formatted_distributions[i] = (demands = demands, probabilities = probabilities)
    end
    
    return formatted_distributions
end

println("Functions get_station_distributions, get_point_predictions, and convert_to_optimization_format defined")

Functions get_station_distributions, get_point_predictions, and convert_to_optimization_format defined


In [ ]:
# Cost parameters (adjust as needed)
q = 2      # Cost to move one bicycle
p_empty = 50.0  # Cost of having an empty station
p_full = 50.0   # Cost of having a full station

# Get unique stations and create mapping
unique_stations = sort(unique(df_test[:, :loc_id]))
N = length(unique_stations)

# Create dictionary from loc_id to index
loc_id_to_idx = Dict(unique_stations[i] => i for i in 1:N)

# Load C (vector N) from capacities.csv
capacity_dict = Dict()
for row in eachrow(capacities)
    capacity_dict[row.loc_id] = row[Symbol("Total Docks")]
end

# Create vector C sorted according to unique_stations
C = [capacity_dict[station] for station in unique_stations]

println("Number of stations: $N")
println("Capacities: $C")

# Get unique days and hours in test
unique_days_test = sort(unique(df_test[:, :day]))
unique_hours_test = sort(unique(df_test[:, :hour]))

println("\nUnique days in test: $(length(unique_days_test))")
println("Unique hours: $(length(unique_hours_test))")

# Cost accumulators per strategy
costs_no_redistribution = Float64[]
costs_with_distributions = Float64[]
costs_with_point_predictions = Float64[]

# Counter for processed iterations
iterations_processed = 0

println("\nProcessing test days and hours...")

for day in unique_days_test
    for hour in unique_hours_test
        # Filter data for this day and hour
        test_subset = filter(row -> row.day == day && row.hour == hour, df_test)
        stock_subset = filter(row -> row.day == day && row.hour == hour, initial_stock)
        
        # Verify that we have data for all stations
        if nrow(test_subset) < N || nrow(stock_subset) < N
            continue
        end
        
        # Calculate S (vector N) from stock.csv
        S = zeros(Float64, N)
        for row in eachrow(stock_subset)
            if haskey(loc_id_to_idx, row.loc_id)
                idx = loc_id_to_idx[row.loc_id]
                S[idx] = row.stock
            end
        end
        
        # Get net_demand (d_real) from test (vector N)
        d_real = zeros(Float64, N)
        for row in eachrow(test_subset)
            if haskey(loc_id_to_idx, row.loc_id)
                idx = loc_id_to_idx[row.loc_id]
                d_real[idx] = row.net_demand
            end
        end
        
        # Calculate cost without redistribution
        cost_no_redist = calculate_cost_no_redistribution(S, d_real, C, p_empty, p_full, N)
        push!(costs_no_redistribution, cost_no_redist)
        
        # Process Station distribution (vector(vector(tuple)) N)
        station_distributions = get_station_distributions(
            day, hour, models, df_train, df_test, unique_stations, 
            feature_cols, categorical_cols, C
        )
        
        # Convert to format expected by solve_prescriptive_optimization
        station_distributions_formatted = convert_to_optimization_format(station_distributions)
        
        # Solve optimization with distributions
        x_dist, _ = solve_prescriptive_optimization(
            S, station_distributions_formatted, C, q, p_empty, p_full, N
        )
        
        # Calculate cost with redistribution using distributions
        cost_with_dist = calculate_cost_with_redistribution(
            S, x_dist, d_real, C, q, p_empty, p_full, N
        )
        push!(costs_with_distributions, cost_with_dist)
        
        # Calculate point predictions
        point_predictions = get_point_predictions(
            day, hour, models, df_test, unique_stations, 
            feature_cols, categorical_cols, C
        )
        
        # Solve optimization with point predictions
        x_point, _ = solve_prescriptive_optimization(
            S, point_predictions, C, q, p_empty, p_full, N
        )
        
        # Calculate cost with redistribution using point predictions
        cost_with_point = calculate_cost_with_redistribution(
            S, x_point, d_real, C, q, p_empty, p_full, N
        )
        push!(costs_with_point_predictions, cost_with_point)
        
        iterations_processed += 1
        
        if iterations_processed % 50 == 0
            println("Processed $iterations_processed iterations...")
        end
    end
end

println("\nProcessing completed. Total iterations: $iterations_processed")
  

Número de estaciones: 10
Capacidades: [53, 19, 31, 19, 19, 35, 23, 23, 19, 15]

Días únicos en test: 396
Horas únicas: 24

Procesando días y horas de test...
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-18
Set parameter Username


Excessive output truncated after 524353 bytes.

Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-18


In [39]:

println("\n" * "="^60)
println("COST COMPARISON")
println("="^60)

avg_cost_no_redist = mean(costs_no_redistribution)
avg_cost_with_dist = mean(costs_with_distributions)
avg_cost_with_point = mean(costs_with_point_predictions)

println("\nAverage cost WITHOUT redistribution: $(round(avg_cost_no_redist, digits=2))")
println("Average cost WITH redistribution (point predictions): $(round(avg_cost_with_point, digits=2))")
println("Average cost WITH redistribution (weighted average cost): $(round(avg_cost_with_dist, digits=2))")

println("\nImprovement with point predictions: $(round(avg_cost_no_redist - avg_cost_with_point, digits=2)) ($(round(100 * (avg_cost_no_redist - avg_cost_with_point) / avg_cost_no_redist, digits=1))%)")
println("Improvement with weighted average cost: $(round(avg_cost_no_redist - avg_cost_with_dist, digits=2)) ($(round(100 * (avg_cost_no_redist - avg_cost_with_dist) / avg_cost_no_redist, digits=1))%)")



COST COMPARISON

Average cost WITHOUT redistribution: 177.93
Average cost WITH redistribution (point predictions): 62.98
Average cost WITH redistribution (weighted average cost): 44.54

Improvement with point predictions: 114.95 (64.6%)
Improvement with weighted average cost: 133.39 (75.0%)
